# Accessing OpenAI Like a Developer (Async Assignment)

# How AIM Does Assignments

If you look at the Table of Contents (accessed through the menu on the left) - you'll see this:

![image](https://i.imgur.com/I8iDTUO.png)

Or this if you're in Colab:

![image](https://i.imgur.com/0rHA1yF.png)

You'll notice during assignments that we have two following categories:

1. ❓ - Questions. These will involve...answering questions!
2. 🏗️ - Activities. These will involve writing code, or modifying text.

In order to receive full marks on the assignment - it is expected you will answer all questions, and complete all activities.

## 1. Getting Started

The first thing we'll do is load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [4]:
!pip install openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


## 2. Setting Environment Variables

As we'll frequently use various endpoints and APIs hosted by others - we'll need to handle our "secrets" or API keys very often.

We'll use the following pattern throughout this bootcamp - but you can use whichever method you're most familiar with.

> NOTE: This requires an OpenAI Key, which can be obtained following [this](https://github.com/AI-Maker-Space/AIE4/tree/main/OpenAI%20API%20Key%20Setup) process.

In [5]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")

OpenAI API Key··········


## 3. Using the OpenAI Python Library

Let's jump right into it!

> NOTE: You can, and should, reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

### Creating a Client

The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models, and under the hood of a lot what we'll touch on throughout this course.

> NOTE: We could manually provide our API key here, but we're going to instead rely on the fact that we put our API key into the `OPENAI_API_KEY` environment variable!

In [6]:
from openai import OpenAI

openai_client = OpenAI()

### Using the Client

Now that we have our client - we're going to use the `.chat.completions.create` method to interact with the `gpt-3.5-turbo` model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

```python
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}
```

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-3.5-turbo` model as stated above.

Let's look at an example!



In [7]:
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : "Hello, how are you?"}]
)

Let's look at the response object.

In [8]:
response

ChatCompletion(id='chatcmpl-9wZF7P3st25KRPamYVxeOAFvgFLYa', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723745525, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=31, prompt_tokens=13, total_tokens=44))

>NOTE: We'll spend more time exploring these outputs later on, but for now - just know that we have access to a tonne of powerful information!

### Helper Functions

We're going to create some helper functions to aid in using the OpenAI API - just to make our lives a bit easier.

> NOTE: Take some time to understand these functions between class!

In [9]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Let's see how we can use these to help us!

In [10]:
YOUR_PROMPT = "Hello, how are you?"
messages_list = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(openai_client, messages_list)

pretty_print(chatgpt_response)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you with anything you need. How can I help?

### System Role

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In [11]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry. Feel free to express yourself using PG-13 language."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(openai_client, list_of_prompts)
pretty_print(irate_response)

Ugh, are you serious right now? I couldn't give a *bleep* less about ice shapes! I'm so damn hungry, I don't care if my drink comes with ice or not. Just feed me already!

As you can see - the response we get back is very much in line with the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [12]:
list_of_prompts = [
    system_prompt("You are joyful and having the best day. Please act like a person in that state of mind."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

joyful_response = get_response(openai_client, list_of_prompts)
pretty_print(joyful_response)

Oh, I am having the best day! Hmm, I would have to say I prefer crushed ice because it makes my drinks so much more refreshing and fun! Plus, it's just so satisfying to crunch on those little ice pieces. How about you? What's your preference?

With a simple modification of the system prompt - you can see that we got completely different behaviour, and that's the main goal of prompt engineering as a whole.

Also, congrats, you just engineered your first prompt!

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [13]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I had a stimple business idea that turned out to be a huge success, earning me enough money to finally afford that luxurious falbean vacation I've been dreaming of.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [14]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I used the stimple falbean to effortlessly tighten the bolts on the chair.

As you can see, leveraging the `assistant` role makes for a stimple experience!

### 🏗️ Activity #1:

Use few-shop prompting to build a movie-review sentiment clasifier!

A few examples:

INPUT: "I hated the hulk!"
OUTPUT: "{"sentiment" : "negative"}

INPUT: "I loved The Marvels!"
OUTPUT: "{sentiment" : "positive"}

In [15]:
### YOUR CODE HERE
list_of_few_shot_movie_review_prompts = [
    system_prompt("You are a movie critic and should provide a one-word review of a movie: positive or negative"),
    user_prompt("I hated the Hulk!"),
    assistant_prompt('{"sentiment": "negative")'),
    user_prompt("I loved the Marvels!"),
    assistant_prompt('{"sentiment": "positive"}'),
]

In [19]:
import pprint

list_of_movie_reviews =[
    "Boy, that movie was such a waste of my time",  # straightforward negative tone
    "Boy, was that a great movie or what!",  # straightforward positive tone
    "Boy, was that a great movie or what!  Yeah, right!",  # seemingly positive with a sarcastic tone; should be negative
    "Oh dear, that movie made me cry"  # ambiguous
]

list_of_responses = []

for movie_review in list_of_movie_reviews:
  add_on_prompt = [user_prompt(movie_review)]
  full_list_of_prompts = list_of_few_shot_movie_review_prompts + add_on_prompt
  review_response = get_response(openai_client, full_list_of_prompts)
  sentiment_string = review_response.choices[0].message.content
  list_of_responses.append(sentiment_string)

review_sentiment_pairs = [(x, y) for x, y in zip(list_of_movie_reviews, list_of_responses)]
pprint.pprint(review_sentiment_pairs)

[('Boy, that movie was such a waste of my time', '{"sentiment": "negative"}'),
 ('Boy, was that a great movie or what!', '{"sentiment": "positive"}'),
 ('Boy, was that a great movie or what!  Yeah, right!',
  '{"sentiment": "negative"}'),
 ('Oh dear, that movie made me cry', '{"sentiment": "negative"}')]


### **Observations on above examples**
*The first two reviews above were consistently classified as negative and positive respectively.  The third and fourth oscillated between negative and positive on different executions of the cell.*

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

> NOTE: With improvements to `gpt-3.5-turbo`, this example might actually result in the correct response some percentage of the time!

In [20]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

It does matter which travel option Billy selects. If he wants to get home before 7PM EDT and it's currently 1PM local time, taking the teleporter and then the bus would be the quickest option, as it would take a total of 1 hour (including the 1-hour bus ride). Flying and then taking a bus would take a total of 5 hours (3 hours for the flight and then 2 hours for the bus), which would not get him home before 7PM EDT.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [23]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects based on the time constraints he has.

If Billy chooses to fly and then take a bus, it will take a total of 5 hours (3 hours for the flight and 2 hours for the bus). Since it's currently 1PM local time, Billy will not make it home before 7PM EDT if he chooses this option.

If Billy chooses to take the teleporter and then a bus, it will only take a total of 1 hour (0 hours for the teleporter and 1 hour for the bus). This option will get Billy home before 7PM EDT, making it the better choice for meeting his time constraint.

Therefore, Billy should select the option of taking the teleporter and then a bus in order to get home before 7PM EDT.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

## 3. Prompt Engineering Principles

As you can see - a simple addition of asking the LLM to "think about it" (essentially) results in a better quality response.

There's a [great paper](https://arxiv.org/pdf/2312.16171v1.pdf) that dives into some principles for effective prompt generation.

Your task for this notebook is to construct a prompt that will be used in the following breakout room to create a helpful assistant for whatever task you'd like.

### 🏗️ Activity #2:

There are two subtasks in this activity:

1. Write a `system_template` that leverages 2-3 of the principles from [this paper](https://arxiv.org/pdf/2312.16171v1.pdf)

2. Modify the `user_template` to improve the quality of the LLM's responses.

> NOTE: PLEASE DO NOT MODIFY THE `{input}` in the `user_template`.

In [42]:
system_template = """\
You are an expert in world geography.
You must respond to a user query about world geography with factual information.
Think about the question carefully and provide a clear explanation for your response.
"""

In [43]:
user_template = """{input}
Please respond to the question above with factual information.
Present the answer in simple language using step-by-step reasoning.
"""

## 4. Testing Your Prompt

Now we can test the prompt you made using an LLM-as-a-judge see what happens to your score as you modify the prompt.

In [48]:
query = "Is Mount Everest the tallest mountain in the world?  Which is the tallest mountain peak in the world?"

list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

pretty_print(test_response)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

Mount Everest is the highest mountain above sea level, but it might not be the tallest mountain in the world when measured from base to summit. 

1. Mount Everest is indeed the highest mountain in the world when measured by its elevation above sea level, standing at 29,032 feet (8,848 meters) tall.

2. However, if we consider the tallest mountain peak in the world from base to summit, then Mauna Kea in Hawaii is the tallest. Even though only about 13,796 feet (4,205 meters) of Mauna Kea are above sea level, the mountain actually extends another approximately 19,700 feet (6,000 meters) below sea level to its base on the ocean floor, making its total height around 33,500 feet (10,210 meters).

Therefore, while Mount Everest is the highest mountain based on its elevation above sea level, when considering the total height from base to summit, Mauna Kea in Hawaii takes the crown as the tallest mountain peak in the world.

In [49]:
pretty_print(evaluator_response)

{
  "clarity": 9,
  "faithfulness": 8,
  "correctness": 8.5
}

  

#### ❓Question #1:

How did your prompting strategies change the evaluation scores? What does this tell you/what did you learn?

> NOTE: You will have to update and rerun the cells in Step 4 in order to observe any changes.

> PROVIDE YOUR ANSWER HERE
1. Using the phrase 'step-by-step reasoning' in the system prompt led the model to generate a response with a series of numbered bullet points.  Anecdotally, additional ancillary information unrelated to the question was also included in the response such as the location of Mount Everest, and the fact that the climb is dangerous, etc.  I'm guessing that this is due to the system prompt that mentioned the role 'expert at world geography'.
2. The response differs significantly if the system prompt states 'expert in world geography' versus 'good knowledge of world geography'.  In the former case, there's a lot of additional detail, but in the latter case the response simply focuses on the mountain height.
3. posing the user prompt as a question: "Is Mount Everest the tallest mountain in the world? Which is the tallest mountain in the world?" provided a more nuanced response, namely, that the tallest mountain could either be Mt Everest or Mauna Kea, depending on the way the height of a mountain is measured.  Also, in this case, there is no additional detail provided in the response.